# Практика 1

На этой практике мы посмотрим на временные ряды, попробуем модели из лекции, и
начнем соревнование!

Для начала склонируем наш репозиторий:

In [ ]:
!git clone https://github.com/ArChanDDD/TS-MCSSummer-2023.git

# Шаг 1: Pandas

В этой главе мы познакомимся с очень полезной для ML библиотекой - `pandas`. Ее смысл - создавать датасеты и предоставлять функционал для работы с ними.

In [ ]:
# Импортируем ее
import pandas as pd
# А эта библиотека для красивых графиков
import matplotlib.pyplot as plt
# А это чтобы крутые операции быстро делать
import numpy as np

Все это время мы будем работать со датасетом `DailyDelhiClimate` - он содержит информацию о погодных условиях в Дели в 2013-2017 годах.

Сначала стоит посмотреть, что там вообще есть:

In [ ]:
df = pd.read_csv('TS-MCSSummer-2023/data/DailyDelhiClimateTrain.csv')

In [ ]:
df.head()

Читать его следует так:
* строка это запись
* столбец это параметр

Получается, из параметров у нас есть

* Дата
* Средняя температура
* Влажность
* Скорость ветра
* Среднее давление

Пример понимания датасета: 1 января 2013 года средняя температура была 10 градусов, влажность - 84.5, ветра не было, среднее давление - 1015 мм.рт.ст.

---

А теперь давайте разберемся, какие операции с датасетом можно делать (хотя корректнее его называть ДатаФрейм):

In [ ]:
# Cложение столбцов
(df['meantemp'] + df['humidity'])[:5]

Как мы видим, он возвращает странную штуку - но можно сделать так:

In [ ]:
list(df['meantemp'] + df['humidity'])[:5]

и вот это уже обычный список - а с ним можно спокойно работать :)

Можно кстати создать новый столбец из списка:

In [ ]:
df['meantemp + humidity'] = list(df['meantemp'] + df['humidity'])
df

Здорово, да?

Помимо этого, столбцы аналогично поддерживают вычитание, деление, умножение, а также можно делать все эти действия с константой:

In [ ]:
(df['meantemp'] * 2)[:5]

Давай чуть-чуть попрактикуемся - посчитай для первых пяти записей разность удвоенной температуры и скорости ветра:

In [ ]:
# TODO

result = ...

assert len(result) == 5, "Нужно посчитать только для первых 5 записей"
assert np.allclose(list(result), [20.0, 11.82, 9.7, 16.1, 8.3]), "Что-то посчитано не так"

Иногда появляется необходимость терироваться по записям в датасете - и, к сожалению, в пандасе с этим все грустно.
Но есть иной вариант:

In [ ]:
# Обернем датасет в массив из numpy
np.array(df[:5])

Как видно, теперь это обычный список из списков, каждый из которых соответствует одной записи. А по списку итерироваться уже не так и сложно!

Следующая задачка - посчитать сумму всех температур с помощью цикла `for`.

In [ ]:
# TODO

result = ...

assert np.isclose(result, 37274.45)

Но вообще эту задачу можно было решить и так:

In [ ]:
result_2 = sum(df['meantemp'])

print(f'Было  - {result}')
print(f'Стало - {result_2}')

На этом вроде все, давайте уже модельки строить!

# Шаг 2: Обзор данных

Раз уж мы там в датасет напихали всякого, давайте заново его импортируем



In [ ]:
df = pd.read_csv('TS-MCSSummer-2023/data/DailyDelhiClimateTrain.csv')
df.head()

На самом деле тут есть проблемка, которую мы сейчас пофиксим:

In [ ]:
df['date']

Как мы видим, тип стоит `object` - это плохо, потому что графики будут кривыми (можете построить, если интересно). Но это фиксится просто:

In [ ]:
from datetime import datetime

df['date'] = [datetime.strptime(x, '%Y-%m-%d') for x in list(df['date'])]

df['date']

Теперь хорошо!

**Наша главная цель** - научиться предсказывать температуру, поэтому, чтобы не забыть, создадим столбец `target`, который будет равен столбцу `meantemp`.

In [ ]:
df['target'] = df['meantemp']

А теперь давайте вообще посмотрим на наш временной ряд:

In [ ]:
_, _ = plt.subplots(1,1,figsize=(20,10))
_ = plt.plot(df['date'], df['target']) # это значит, что по оси ОХ будет дата, а по оси OY будет наша температура

Первое правило при работе с любой задачей машинного обучения - понять, что за данные. Что мы можем сказать про этот ряд? Есть ли у него тренд? А периодичность? Можно его назвать стационарным?

# Шаг 3: Наивная модель

Понимаю, хочется начать предсказывать - мы же тут как раз для этого!

Поэтому, давайте начнем с базовой модели - наивной. На всякий случай напомню, что это:

**Наивной моделью** для задачи прогнозирования временных рядов называется модель, которая сопоставляет $y_t$ значение, которое было $lag$ шагов назад - то есть $y_{t-lag}$

Для оценки наших моделей мы будем использовать валидацию на последнем "горбу" - вот этой четвертой горке на графике.

Для этого мы разобьем датасет на две части - на одной мы будем обучаться, а на другой проверять качество наших моделек.

In [ ]:
val_start = 1117

df_train = df[:val_start]
df_val = df[val_start:]

_, _ = plt.subplots(1,1,figsize=(20,10))
_ = plt.plot(df_train['date'], df_train['target'], label='train')
_ = plt.plot(df_val['date'], df_val['target'], label='val')
_ = plt.legend()

In [ ]:
# Просто функция для построения графиков
def plot_graph(y_pred):
  _, _ = plt.subplots(1,1,figsize=(18,10))
  _ = plt.plot(df_train['date'], df_train['target'], label='train')
  _ = plt.plot(df_val['date'], df_val['target'], label='val')
  _ = plt.plot(df_val['date'], y_pred, label='predict')
  _ = plt.legend()

In [ ]:
lag = len(df_val) # пусть будет так

# TODO
# Необходимо для каждого i < len(df_val) добавить в y_pred то значение, которое было в df_train lag позиций назад

y_pred = []

for i in range(len(df_val)):
  ...


plot_graph(y_pred)

Вначале вроде как даже неплохо, но потом становится совсем плохо...

Давайте оценим насколько все плохо - для этого посчитаем функцию
$$
MSE = \frac{1}{n}\sum_{i=0}^n (y_i - \hat{y}_i)^2
$$

Где:

* $y_i$ - правильно значение (которое `val`)
* $\hat{y}_i$ - предсказанное значение (которое `y_pred`)
* $n$ - длина массива `y_pred` и `val`

Реализуйте эту функцию!

In [ ]:
# TODO

def MSE(y_true, y_pred):
  result = ...
  return result

In [ ]:
mse = MSE(list(df_val['target']), y_pred)
print(mse)

assert np.isclose(mse, 17.575336), "Функция делает что-то не так..."

Сложно сказать, хорошо это или нет, но будем считать эту ошибку ориентиром. Наша цель - сделать ее как можно меньше (а значит предсказание станет наиболее правильным).

# Шаг 4: Линейная регрессия

Наш следующий шаг - построить линейную регрессию. К огромному счастью, в питоне это все уже есть, а значит наша задача лишь подготовить данные

## Регрессия на одном параметре

Для начала давайте попробуем, как в примере на лекции, построить регрессию, в которой `x` будет равен дню в месяце.

<details><summary> Подсказка 1 </summary>

Не забывайте о том, что по `df_train['date']` можно итерироваться!

</details>

<details><summary> Подсказка 2 </summary>

Можно сделать для каждого `x` в `df_train['date']` можно сделать `x.day` и получить день месяца.

</details>

In [ ]:
# Достаем модель - линейную регрессию
from sklearn.linear_model import LinearRegression

# Почти у всех основных моделей в ML есть основные методы:
# * fit() - обучение модели, на вход как правило подается X - данные, y - целевое значение
# * predict() - предсказание, на вход подается X - возвраещается y_pred

# Определяем модель
model = LinearRegression()

# TODO
# X и X_val строятся одинаковым алгоритмом, но на разных датасетах

y = list(df_train['target'])
X = ...
X_val = ...


# Тут сначала оборачиваем в массив numpy, потом делаем так, чтобы числа были в столбик, а не в строку
X = np.array(X).reshape(-1, 1)
X_val = np.array(X_val).reshape(-1, 1)
# Ну и обучаем, конечно
model.fit(X, y)

y_pred = model.predict(X_val)

plot_graph(y_pred)

Получилось что-то странное, не так ли? И в этом на самом деле нет ничего странного - насколько мы помним, наша функция принимала вид $y=kx$, а даже при учете, что дни чередуются, описать что-то не получится :(

В любом случае, посчитаем MSE

In [ ]:
MSE(y_pred, list(df_val['target']))

Cурово. Попробуем сделать регрессию на нескольких параметрах - построим $y=\sum k_ix_i$

## Регрессия на нескольких параметрах

Вообще, по графику можно увидеть, что линейной функцией приблизить не получится - вот бы можно было сделать что-то вроде $y=k_1x_1 + k_2x_2^2$...

Но ведь так можно сделать! С одно стороны, это уже полиномиальная регрессия, с другой стороны - это линейная регрессия, в которой есть полиномы. Поэтому давайте создадим эти полиному - сделайте аналогично прыдущему пункту, но теперь мы будем обучаться не на `[day]`, а на `[day, day^2, month, month^2]`

In [ ]:
model = LinearRegression()

# TODO

y = list(df_train['target'])
X = ...
X_val = ...


# Тут сначала оборачиваем в массив numpy, потом делаем так, чтобы числа были в столбик, а не в строку
X = np.array(X).reshape(-1, 4)
X_val = np.array(X_val).reshape(-1,4)
# Ну и обучаем, конечно
model.fit(X, y)

y_pred = model.predict(X_val)

plot_graph(y_pred)

In [ ]:
MSE(y_pred, list(df_val['target']))

Ну это ведь уже совсем другое дело! Даже на графике видно, что стало заметно лучше, а MSE стал лучше почти на 40%! И все благодаря тому, что мы добавили несколько экзогенных переменных.

На самом деле, Линейная регрессия - штука очень мощная. И не просто так всякие дата-саентисты часто ей пользуются - ведь она умеет так:

In [ ]:
list(model.coef_)

Это коэфициенты нашей модели - $i$-й коэфициент соответсвует $i$-му параметру в $X$.

Видно, что модель обращает большое внимание на месяц, а не на день - именно поэтому наша предыдущая модель была такой плохой.

---

Не думайте, что линейная регрессия это не тру - ради интереса можете попробовать добавить еще параметров. Вот увидите, результат может быть очень крутым 😁

А мы идем дальше к самой крутой моделе в лекции

# Шаг 5: ARIMA

Пришло время вспомнить, что это такое, и почему нам тут стоит использовать именно `ARIMA`, а не `ARMA` (или нет?)

Поскольку `ARIMA` из разряда чуть выше, чем линейная регрессия, в ней методы будут уже несколько другими. В ней помимо метода `predict` есть метод `forecast` - потому что именно так называют предсказание временного ряда. В то же время `predict` предсказывает значения для *уже известных* данных

Посмотрим, как она работает:

p.s. а работает она заметно дольше

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Order идет в следующем порядке - p, d, q - учтите это

model = ARIMA(
    endog=df_train.target,
    exog=None,
    order=(10, 0, 12),
    seasonal_order=(0, 0, 0, 0),
    trend=None,
    enforce_stationarity=True,
    enforce_invertibility=True,
    concentrate_scale=False,
    trend_offset=1,
    dates=None,
    freq=None,
    missing='none',
    validate_specification=True
    )
modelfit = model.fit()

y_pred = modelfit.forecast(steps=len(df_val))

plot_graph(y_pred)


In [ ]:
MSE(list(df_val['target']), list(y_pred))

На самом деле, не так плохо - но кажется, что у линейно регрессии ошибка была сильно меньше. Значит ли это, что регрессия гораздо лучше чем ARIMA?

Cпойлер - нет. Регрессия не имеет никаких параметров, все что она делает - минимизирует функцию. А вот ARIMA в зависимости от параметров `p`, `q` и `d` может давать очень разные результаты. Имеет смысл попрактиковаться!

In [ ]:
# Сюда можно скопировать код с АРИМой и попробовать сделать ее лучше

# Cоревнование

Наш следующий этап - понять, насколько вообще наши предсказания хорошие.

Для этого мы пойдем по пути крутых ML-разработчиков, и обратимся к **Kaggle**.

Cсылка на наше соревнование - https://www.kaggle.com/t/91f9d8ed4419473dad88151c5e7411f3

Надеемся, к этому моменту преподаватель уже рассказал, как им пользоваться, но если нет - расскажем тут.

1.   Для начала нужно будет зарегистрироваться в `kaggle` - можно войти через гугл, это не особо важно.
2.   По старой методике делаем `y_pred`, но теперь не для `df_val`, а для `df_test`, и запускаем функцию `make_submission_file`. Если все ок - качаем файлик на компьютер и идем на страничку соревнования в `kaggle`.
3.   На странице соревнования жмем большую синюю кнопку `Make Submission` и заливаем туда наш файлик.
4.   Ждем результата и думаем как сделать его еще лучше!
5.   (не обязательно) В случае проблем или вопросов пишем преподавателям 🎃



In [ ]:
df_test = pd.read_csv('TS-MCSSummer-2023/data/DailyDelhiClimateTest.csv')
df_test = df_test[['date', 'humidity', 'wind_speed', 'meanpressure']]
old_dates = list(df_test['date'])
df_test['date'] = [datetime.strptime(x, '%Y-%m-%d') for x in df_test['date']]
df_test

In [ ]:
# Функция для проверки корректности перед отправкой

def check_submission(df_to_check):
  assert str(type(df_to_check)) == "<class 'pandas.core.frame.DataFrame'>", "Это не датафрейм"
  assert len(df_to_check) == 114, "Длина датафрейма должна быть 114"
  assert list(df_to_check.columns) == ['date', 'target'], "У датафрейма должны быть колонки ['date', 'target']"

  df_to_check.set_index('date').to_csv('submission.csv')

def make_submission_file(y_pred):
  assert len(y_pred) == 114, "Длина датафрейма должна быть 114"

  df_sub = pd.DataFrame(data=np.array([old_dates, y_pred]).transpose(), columns=['date', 'target'])
  check_submission(df_sub)

In [ ]:
# Все что вы захотите!